In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import gc
import numpy as np

import tensorflow as tf

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import merlin.models.tf as mm

2023-03-10 13:19:41.332031: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2023-03-10 13:19:43.702598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-10 13:19:43.703049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-10 13:19:43.703227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgr

In [2]:
train = Dataset("ecom_dataset/0001/train.parquet")
valid = Dataset("ecom_dataset/0002/valid.parquet")

In [3]:
target = 'sess_pid_seq'

In [4]:
# a couple of starter hyperparams

d_model = 192
n_layer = 3
n_head = 16
batch_size = 128
learning_rate = 0.0006667377132554976
n_epoch = 5

In [5]:
def get_model():
    mlp_block = mm.MLPBlock(
                    [128,d_model],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    from merlin.schema.io.tensorflow_metadata import TensorflowMetadata

    schema = TensorflowMetadata.from_proto_text_file(
        '../',
        file_name='rees46_schema_modified.pbtxt'
    ).to_merlin_schema()

    train.schema = schema

    input_block = mm.InputBlockV2(
        train.schema.select_by_name('sess_pid_seq'),    
        embeddings=mm.Embeddings(
            train.schema.select_by_name('sess_pid_seq'), 
            sequence_combiner=None,
            dim=d_model
            ),
    #     pre=mm.StochasticSwapNoise()
    )

    train.schema = train.schema.select_by_name('sess_pid_seq')

    xlnet_block = mm.XLNetBlock(d_model=d_model, n_head=n_head, n_layer=n_layer)

    dense_block = mm.SequentialBlock(
        input_block,
        mlp_block,
        xlnet_block
    )

    mlp_block2 = mm.MLPBlock(
                    [128,d_model],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    prediction_task = mm.CategoricalOutput(
        to_call=input_block["categorical"][target],
    )

    model_transformer = mm.Model(dense_block, mlp_block2, prediction_task)

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
    )

    model_transformer.compile(run_eagerly=False, optimizer=optimizer, loss="categorical_crossentropy",
                  metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[10])
                 )
    return model_transformer, xlnet_block

# Run 1

In [6]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/5


2023-03-10 13:19:51.258201: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model/sequential_block_4/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 83s 110ms/step - loss: 8.9114 - recall_at_10: 0.0409 - mrr_at_10: 0.0151 - ndcg_at_10: 0.0211 - map_at_10: 0.0151 - precision_at_10: 0.0041 - regularization_loss: 0.0000e+00 - loss_batch: 8.9101
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 5.7403 - recall_at_10: 0.2994 - mrr_at_10: 0.1857 - ndcg_at_10: 0.2126 - map_at_10: 0.1857 - precision_at_10: 0.0299 - regularization_loss: 0.0000e+00 - loss_batch: 5.7358
Epoch 3/5
677/677 [==============================] - 75s 110ms/step - loss: 3.8640 - recall_at_10: 0.5785 - mrr_at_10: 0.4536 - ndcg_at_10: 0.4835 - map_at_10: 0.4536 - precision_at_10: 0.0579 - regularization_loss: 0.0000e+00 - loss_batch: 3.8700
Epoch 4/5
677/677 [==============================] - 75s 110ms/step - loss: 3.0672 - recall_at_10: 0.6808 - mrr_at_10: 0.5664 - ndcg_at_10: 0.5939 - map_at_10: 0.5664 - precision_at_10: 0.0681 - regularization_loss: 0.0000e+00 - loss_batch: 3.0720
Epoch 5/5
84/84 [=

{'loss': 8.741933822631836,
 'recall_at_10': 0.16701146960258484,
 'mrr_at_10': 0.06411589682102203,
 'ndcg_at_10': 0.08810190856456757,
 'map_at_10': 0.06411589682102203,
 'precision_at_10': 0.016701148822903633,
 'regularization_loss': 0.0,
 'loss_batch': 10.003721237182617}

# Run 2

In [7]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 80s 110ms/step - loss: 9.0406 - recall_at_10: 0.0356 - mrr_at_10: 0.0130 - ndcg_at_10: 0.0183 - map_at_10: 0.0130 - precision_at_10: 0.0036 - regularization_loss: 0.0000e+00 - loss_batch: 9.0326
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 6.0845 - recall_at_10: 0.2649 - mrr_at_10: 0.1565 - ndcg_at_10: 0.1821 - map_at_10: 0.1565 - precision_at_10: 0.0265 - regularization_loss: 0.0000e+00 - loss_batch: 6.0807
Epoch 3/5
677/677 [==============================] - 75s 110ms/step - loss: 4.2293 - recall_at_10: 0.5184 - mrr_at_10: 0.3883 - ndcg_at_10: 0.4194 - map_at_10: 0.3883 - precision_at_10: 0.0518 - regularization_loss: 0.0000e+00 - loss_batch: 4.2323
Epoch 4/5
677/677 [==============================] - 75s 110ms/step - loss: 3.3441 - recall_at_10: 0.6468 - mrr_at_10: 0.5229 - ndcg_at_10: 0.5526 - map_at_10: 0.5229 - precision_at_10: 0.0647 - regularization_loss: 0.0000e+00 - loss_batch: 3.3486
Epoch 5/5
84/84 [=

{'loss': 8.827858924865723,
 'recall_at_10': 0.15591499209403992,
 'mrr_at_10': 0.06090494617819786,
 'ndcg_at_10': 0.08297329396009445,
 'map_at_10': 0.06090494617819786,
 'precision_at_10': 0.01559150218963623,
 'regularization_loss': 0.0,
 'loss_batch': 10.57563304901123}

# Run 3

In [8]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 80s 110ms/step - loss: 8.9635 - recall_at_10: 0.0396 - mrr_at_10: 0.0146 - ndcg_at_10: 0.0204 - map_at_10: 0.0146 - precision_at_10: 0.0040 - regularization_loss: 0.0000e+00 - loss_batch: 8.9589
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 6.2358 - recall_at_10: 0.2417 - mrr_at_10: 0.1391 - ndcg_at_10: 0.1633 - map_at_10: 0.1391 - precision_at_10: 0.0242 - regularization_loss: 0.0000e+00 - loss_batch: 6.2350
Epoch 3/5
677/677 [==============================] - 75s 110ms/step - loss: 4.4725 - recall_at_10: 0.4996 - mrr_at_10: 0.3675 - ndcg_at_10: 0.3991 - map_at_10: 0.3675 - precision_at_10: 0.0500 - regularization_loss: 0.0000e+00 - loss_batch: 4.4748
Epoch 4/5
677/677 [==============================] - 75s 110ms/step - loss: 3.6489 - recall_at_10: 0.6128 - mrr_at_10: 0.4839 - ndcg_at_10: 0.5148 - map_at_10: 0.4839 - precision_at_10: 0.0613 - regularization_loss: 0.0000e+00 - loss_batch: 3.6543
Epoch 5/5
84/84 [=

{'loss': 8.832069396972656,
 'recall_at_10': 0.1426556259393692,
 'mrr_at_10': 0.05639006569981575,
 'ndcg_at_10': 0.07650619745254517,
 'map_at_10': 0.05639006569981575,
 'precision_at_10': 0.01426556333899498,
 'regularization_loss': 0.0,
 'loss_batch': 10.100401878356934}

# Run 4

In [9]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 80s 110ms/step - loss: 9.0739 - recall_at_10: 0.0361 - mrr_at_10: 0.0125 - ndcg_at_10: 0.0180 - map_at_10: 0.0125 - precision_at_10: 0.0036 - regularization_loss: 0.0000e+00 - loss_batch: 9.0756
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 6.4023 - recall_at_10: 0.2372 - mrr_at_10: 0.1349 - ndcg_at_10: 0.1591 - map_at_10: 0.1349 - precision_at_10: 0.0237 - regularization_loss: 0.0000e+00 - loss_batch: 6.4020
Epoch 3/5
677/677 [==============================] - 75s 111ms/step - loss: 4.7934 - recall_at_10: 0.4544 - mrr_at_10: 0.3194 - ndcg_at_10: 0.3516 - map_at_10: 0.3194 - precision_at_10: 0.0454 - regularization_loss: 0.0000e+00 - loss_batch: 4.7958
Epoch 4/5
677/677 [==============================] - 75s 111ms/step - loss: 3.7131 - recall_at_10: 0.5913 - mrr_at_10: 0.4595 - ndcg_at_10: 0.4911 - map_at_10: 0.4595 - precision_at_10: 0.0591 - regularization_loss: 0.0000e+00 - loss_batch: 3.7160
Epoch 5/5
84/84 [=

{'loss': 8.922541618347168,
 'recall_at_10': 0.14425428211688995,
 'mrr_at_10': 0.057682257145643234,
 'ndcg_at_10': 0.077837273478508,
 'map_at_10': 0.057682257145643234,
 'precision_at_10': 0.014425428584218025,
 'regularization_loss': 0.0,
 'loss_batch': 10.947548866271973}

# Run 5

In [10]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 80s 110ms/step - loss: 9.0454 - recall_at_10: 0.0381 - mrr_at_10: 0.0139 - ndcg_at_10: 0.0195 - map_at_10: 0.0139 - precision_at_10: 0.0038 - regularization_loss: 0.0000e+00 - loss_batch: 9.0386
Epoch 2/5
677/677 [==============================] - 75s 110ms/step - loss: 6.3489 - recall_at_10: 0.2430 - mrr_at_10: 0.1385 - ndcg_at_10: 0.1632 - map_at_10: 0.1385 - precision_at_10: 0.0243 - regularization_loss: 0.0000e+00 - loss_batch: 6.3435
Epoch 3/5
677/677 [==============================] - 75s 111ms/step - loss: 4.7853 - recall_at_10: 0.4602 - mrr_at_10: 0.3227 - ndcg_at_10: 0.3555 - map_at_10: 0.3227 - precision_at_10: 0.0460 - regularization_loss: 0.0000e+00 - loss_batch: 4.7868
Epoch 4/5
677/677 [==============================] - 75s 111ms/step - loss: 3.6873 - recall_at_10: 0.6026 - mrr_at_10: 0.4710 - ndcg_at_10: 0.5025 - map_at_10: 0.4710 - precision_at_10: 0.0603 - regularization_loss: 0.0000e+00 - loss_batch: 3.6936
Epoch 5/5
84/84 [=

{'loss': 8.671070098876953,
 'recall_at_10': 0.15074290335178375,
 'mrr_at_10': 0.05898994952440262,
 'ndcg_at_10': 0.08035662025213242,
 'map_at_10': 0.05898994952440262,
 'precision_at_10': 0.015074292197823524,
 'regularization_loss': 0.0,
 'loss_batch': 9.946744918823242}